In [59]:
#!pip install tensorflow --upgrade
#!pip install keras --upgrade

In [60]:
"""
## Setup
"""
import keras
from keras import layers
from keras import ops

import math
import numpy as np
import pandas as pd
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from functools import partial

In [61]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

# root_dir = "/content/gdrive/MyDrive/"

In [62]:
# !unzip gdrive/MyDrive/packet_features.zip

In [63]:
scana = pd.read_csv('packet_features/scan_A.csv')
scansu = pd.read_csv('packet_features/scan_sU.csv')
mqtt_bruteforce = pd.read_csv('packet_features/mqtt_bruteforce.csv')
sparta = pd.read_csv('packet_features/sparta.csv')
normal = pd.read_csv('packet_features/normal.csv')

#data = pd.concat([scana, scansu,mqtt_bruteforce,sparta,normal],axis=0)

/tmp/ipykernel_12469/509904704.py:3: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  mqtt_bruteforce = pd.read_csv('packet_features/mqtt_bruteforce.csv')
/tmp/ipykernel_12469/509904704.py:4: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  sparta = pd.read_csv('packet_features/sparta.csv')
/tmp/ipykernel_12469/509904704.py:5: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  normal = pd.read_csv('packet_features/normal.csv')


In [64]:
scana['is_attack'].value_counts()

is_attack
0    70768
1    40624
Name: count, dtype: int64

In [65]:
scansu['is_attack'].value_counts()

is_attack
0    210819
1     22436
Name: count, dtype: int64

In [66]:
sparta['is_attack'].value_counts()

is_attack
1            19103574
0              917206
1              625369
0               29971
is_attack          20
Name: count, dtype: int64

In [67]:
normal['is_attack'].value_counts()

is_attack
0            1023463
0              32767
is_attack          1
Name: count, dtype: int64

In [68]:
mqtt_bruteforce['is_attack'].value_counts()

is_attack
1            9686086
1             327056
0              31550
0                614
is_attack         10
Name: count, dtype: int64

In [69]:
def pre_processing(data):

    data = data.drop(columns=["timestamp","src_ip","dst_ip", "src_port","dst_port"])
    data['protocol'] = data['protocol'].astype(str)
    data['is_attack'] = data['is_attack'].astype(str)

    non_numeric_columns = data.select_dtypes(include=['object']).columns

    for col in data.columns:
        if col not in ['protocol', 'is_attack']:
            data[col] = pd.to_numeric(data[col], errors='coerce')

    for column in data:
        if column not in ['protocol', 'is_attack']:
            clean_col = data[column].dropna()
            mean = clean_col.mean()
            data[column].fillna(mean, inplace=True)

    data = data.sample(frac=1).reset_index(drop=True)

    data['is_attack'].replace('0', 'no', inplace=True)
    data['is_attack'].replace('1', 'yes', inplace=True)

    data.loc[data['is_attack'] == 'is_attack', 'is_attack'] = 'yes'

    return data

def balance_and_concat(d1,d2,d3,d4,normal):
    normal_final = pd.DataFrame()
    data = pd.DataFrame()
    n = normal.copy()
    n = n[n.is_attack == 'no']
    
    #i had problems trying to do a for loop so i had to do each dataset separately

    ############################## d1 ################################
    d = d1.copy().sort_values(by=['is_attack']).reset_index(drop=True)
    d_yes = (d['is_attack'].values == 'yes').sum()
    d_no = (d['is_attack'].values == 'no').sum()
    print("d1")
    print("yes: ", d_yes)
    print("no: ", d_no)
    if min(d_yes,d_no) == d_no: #there are remaining 'yes' rows to be concatenated with "normal"
        d_final = d[:d_no*2]
        d_remaining_yes = d[d_no*2:]

        if len(n) >= len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n[:len(d_remaining_yes)], d_remaining_yes],axis=0).reset_index(drop=True)
            n = n[len(d_remaining_yes):].reset_index(drop=True)
        elif len(n) > 0 and len(n) < len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n, d_remaining_yes[:len(n)]],axis=0).reset_index(drop=True)
            n = pd.DataFrame()
    else: #there aren't any remaining 'yes' rows to be concatenated with "normal"
        d_final = pd.concat([d[:d_yes],d[-d_yes:]],axis=0)
    data = pd.concat([data,d_final],axis=0)
    
    print("d1_final")
    print("yes: ", (d_final['is_attack'].values == 'yes').sum())
    print("no: ", (d_final['is_attack'].values == 'no').sum())
    ############################## d2 ################################
    d = d2.copy().sort_values(by=['is_attack']).reset_index(drop=True)
    d_yes = (d['is_attack'].values == 'yes').sum()
    d_no = (d['is_attack'].values == 'no').sum()
    print("d2")
    print("yes: ", d_yes)
    print("no: ", d_no)
    if min(d_yes,d_no) == d_no: #there are remaining 'yes' rows to be concatenated with "normal"
        d_final = d[:d_no*2]
        d_remaining_yes = d[d_no*2:]

        if len(n) >= len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n[:len(d_remaining_yes)], d_remaining_yes],axis=0).reset_index(drop=True)
            n = n[len(d_remaining_yes):].reset_index(drop=True)
        elif len(n) > 0 and len(n) < len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n, d_remaining_yes[:len(n)]],axis=0).reset_index(drop=True)
            n = pd.DataFrame()
    else: #there aren't any remaining 'yes' rows to be concatenated with "normal"
        d_final = pd.concat([d[:d_yes],d[-d_yes:]],axis=0)
    data = pd.concat([data,d_final],axis=0)
    
    print("d2_final")
    print("yes: ", (d_final['is_attack'].values == 'yes').sum())
    print("no: ", (d_final['is_attack'].values == 'no').sum())
    ############################## d3 ################################
    d = d3.copy().sort_values(by=['is_attack']).reset_index(drop=True)
    d_yes = (d['is_attack'].values == 'yes').sum()
    d_no = (d['is_attack'].values == 'no').sum()
    print("d3")
    print("yes: ", d_yes)
    print("no: ", d_no)
    if min(d_yes,d_no) == d_no: #there are remaining 'yes' rows to be concatenated with "normal"
        d_final = d[:d_no*2]
        d_remaining_yes = d[d_no*2:]

        if len(n) >= len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n[:len(d_remaining_yes)], d_remaining_yes],axis=0).reset_index(drop=True)
            n = n[len(d_remaining_yes):].reset_index(drop=True)
        elif len(n) > 0 and len(n) < len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n, d_remaining_yes[:len(n)]],axis=0).reset_index(drop=True)
            n = pd.DataFrame()
    else: #there aren't any remaining 'yes' rows to be concatenated with "normal"
        d_final = pd.concat([d[:d_yes],d[-d_yes:]],axis=0)
    data = pd.concat([data,d_final],axis=0)
    
    print("d3_final")
    print("yes: ", (d_final['is_attack'].values == 'yes').sum())
    print("no: ", (d_final['is_attack'].values == 'no').sum())
    ############################## d4 ################################
    d = d4.copy().sort_values(by=['is_attack']).reset_index(drop=True)
    d_yes = (d['is_attack'].values == 'yes').sum()
    d_no = (d['is_attack'].values == 'no').sum()
    print("d4")
    print("yes: ", d_yes)
    print("no: ", d_no)
    if min(d_yes,d_no) == d_no: #there are remaining 'yes' rows to be concatenated with "normal"
        d_final = d[:d_no*2]
        d_remaining_yes = d[d_no*2:]

        if len(n) >= len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n[:len(d_remaining_yes)], d_remaining_yes],axis=0).reset_index(drop=True)
            n = n[len(d_remaining_yes):].reset_index(drop=True)
        elif len(n) > 0 and len(n) < len(d_remaining_yes):
            normal_final = pd.concat([normal_final, n, d_remaining_yes[:len(n)]],axis=0).reset_index(drop=True)
            n = pd.DataFrame()
    else: #there aren't any remaining 'yes' rows to be concatenated with "normal"
        d_final = pd.concat([d[:d_yes],d[-d_yes:]],axis=0)
    data = pd.concat([data,d_final],axis=0)
    
    print("d4_final")
    print("yes: ", (d_final['is_attack'].values == 'yes').sum())
    print("no: ", (d_final['is_attack'].values == 'no').sum())
    
    data = pd.concat([data,n],axis=0)
    n_yes = (normal_final['is_attack'].values == 'yes').sum()
    n_no = (normal_final['is_attack'].values == 'no').sum()
    print("n")
    print("yes: ", n_yes)
    print("no: ", n_no)

    #data = pd.concat([d[1], d[2],d[3],d[4],n],axis=0)
    
    #BALANCE DATASET
    #aux = data.groupby('is_attack')
    #data = aux.apply(lambda x: x.sample(aux.size().min()).reset_index(drop=True))

    data = data.sample(frac=1).reset_index(drop=True)
    #print('quantity of no: ', data['is_attack'].value_counts()[0])
    #print('quantity of yes: ', data['is_attack'].value_counts()[1])
    
    return data

In [70]:
scana = pre_processing(scana)
scansu =  pre_processing(scansu)
mqtt_bruteforce = pre_processing(mqtt_bruteforce)
sparta = pre_processing(sparta)
normal = pre_processing(normal)

data = balance_and_concat(scana, scansu,mqtt_bruteforce,sparta,normal)

/tmp/ipykernel_12469/1843275389.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(mean, inplace=True)
/tmp/ipykernel_12469/1843275389.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

d1
yes:  40624
no:  70768
d1_final
yes:  40624
no:  40624
d2
yes:  22436
no:  210819
d2_final
yes:  22436
no:  22436
d3
yes:  10013152
no:  32164
d3_final
yes:  32164
no:  32164
d4
yes:  19728963
no:  947177
d4_final
yes:  947177
no:  947177
n
yes:  1056230
no:  1056230


In [71]:
#FEATURE SELECTION

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

data = data.sample(frac=1).reset_index(drop=True)
X, y = data.drop(columns=["protocol","is_attack"]).values, data['is_attack'].values
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
importances = clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)

#print('X shape: ', X.shape)
#print('X columns: ', len(data.columns))
#print('X_new shape: ', X_new.shape)
#print('feature importances: ', importances)

selected_indexes = [i+1 for i in range(len(importances)) if importances[i] >= 0.1]
selected_indexes = selected_indexes+[0,len(data.columns)-1]

not_sel_col = [data.columns[i] for i in range(len(data.columns)) if i not in selected_indexes]
sel_col = [data.columns[i] for i in range(len(data.columns)) if i in selected_indexes]

data = data.drop(columns=not_sel_col)

In [72]:
#data = pd.concat([scana, scansu],axis=0)
#data = pd.concat([scana, scansu,mqtt_bruteforce,normal],axis=0)

#data = data.drop(columns=["timestamp","src_ip","dst_ip", "src_port","dst_port"])
#data['protocol'] = data['protocol'].astype(str)
#data['is_attack'] = data['is_attack'].astype(str)

In [73]:
data['is_attack'].value_counts()

is_attack
yes    1042401
no     1042401
Name: count, dtype: int64

In [74]:
data['is_attack'].value_counts()

is_attack
yes    1042401
no     1042401
Name: count, dtype: int64

In [75]:
non_numeric_columns = data.select_dtypes(include=['object']).columns

In [76]:
non_numeric_columns

Index(['protocol', 'is_attack'], dtype='object')

In [77]:
#for col in data.columns:
#    if col not in ['protocol', 'is_attack']:
#      data[col] = pd.to_numeric(data[col], errors='coerce')

In [78]:
data.dtypes

protocol               object
ttl                   float64
ip_len                float64
tcp_flag_push         float64
mqtt_messagetype      float64
mqtt_messagelength    float64
is_attack              object
dtype: object

In [79]:
#for column in data:
#    if column not in ['protocol', 'is_attack']:
#        clean_col = data[column].dropna()
#        mean = clean_col.mean()
#        data[column].fillna(mean, inplace=True)
#
#data = data.sample(frac=1).reset_index(drop=True)
#
#data['is_attack'].replace('0', 'no', inplace=True)
#data['is_attack'].replace('1', 'yes', inplace=True)

In [80]:
#ADD WEIGHTS - 1 FOR ALL SAMPLES
data['fnlwgt'] = 1

In [81]:
data['is_attack'].value_counts()

is_attack
yes    1042401
no     1042401
Name: count, dtype: int64

In [82]:
# Replace 'is_attack' with 'yes' only in rows where the value of 'is_attack' column is 'is_attack'
#data.loc[data['is_attack'] == 'is_attack', 'is_attack'] = 'yes'


In [83]:
data['is_attack'].value_counts()

is_attack
yes    1042401
no     1042401
Name: count, dtype: int64

In [84]:
##BALANCE DATASET
#
#aux = data.groupby('is_attack')
#data = aux.apply(lambda x: x.sample(aux.size().min()).reset_index(drop=True))
#
#data = data.sample(frac=1).reset_index(drop=True)
#print('quantity of no: ', data['is_attack'].value_counts()[0])
#print('quantity of yes: ', data['is_attack'].value_counts()[1])

In [85]:
data_train = data.loc[:int(0.8*len(data))]
data_test = data.loc[int(0.8*len(data)):(int(0.8*len(data))+int(0.1*len(data)))]
data_valid = data.loc[(int(0.8*len(data))+int(0.1*len(data))):]

train_data_file = "train_data.csv"
test_data_file = "test_data.csv"
valid_data_file = "val_data.csv"

data_train.to_csv(train_data_file, index=False, header=False)
data_test.to_csv(test_data_file, index=False, header=False)
data_valid.to_csv(valid_data_file, index=False, header=False)

In [86]:
import time
from sklearn.metrics import classification_report, roc_auc_score


In [87]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


## Prepare the data

data_columns = list(data.columns)

CSV_HEADER = data_columns


"""
## Define dataset metadata

Here, we define the metadata of the dataset that will be useful for reading and parsing
the data into input features, and encoding the input features with respect to their types.
"""

# A list of the numerical feature names.
NUMERIC_FEATURE_NAMES = data_columns[1:len(data_columns)-2]


# A dictionary of the categorical features and their vocabulary.
CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "protocol": sorted(list(data_train["protocol"].unique())),
}
# Name of the column to be used as instances weight.
WEIGHT_COLUMN_NAME = "fnlwgt"
# A list of the categorical feature names.
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
# A list of all the input features.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
# A list of column default values for each feature.
COLUMN_DEFAULTS = [
    [0.0] if feature_name in NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME] else ["NA"]
    for feature_name in CSV_HEADER
]
# The name of the target feature.
TARGET_FEATURE_NAME = "is_attack"
# A list of the labels of the target features.
TARGET_LABELS = ["yes", "no"]

"""
## Configure the hyperparameters

The hyperparameters includes model architecture and training configurations.
"""

LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
DROPOUT_RATE = 0.2
BATCH_SIZE = 265
NUM_EPOCHS = 20

NUM_TRANSFORMER_BLOCKS = 4  # Number of transformer blocks.
NUM_HEADS = 4  # Number of attention heads.
EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
MLP_HIDDEN_UNITS_FACTORS = [
    2,
    1,
]  # MLP hidden layer units, as factors of the number of inputs.
NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

"""
## Implement data reading pipeline

We define an input function that reads and parses the file, then converts features
and labels into a[`tf.data.Dataset`](https://www.tensorflow.org/guide/datasets)
for training or evaluation.
"""

target_label_lookup = layers.StringLookup(
    vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
)


def prepare_example(features, target):
    target_index = target_label_lookup(target)
    weights = features.pop(WEIGHT_COLUMN_NAME)
    return features, target_index, weights


lookup_dict = {}
for feature_name in CATEGORICAL_FEATURE_NAMES:
    vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
    # Create a lookup to convert a string values to an integer indices.
    # Since we are not using a mask token, nor expecting any out of vocabulary
    # (oov) token, we set mask_token to None and num_oov_indices to 0.
    lookup = layers.StringLookup(
        vocabulary=vocabulary, mask_token=None, num_oov_indices=0
    )
    lookup_dict[feature_name] = lookup


def encode_categorical(batch_x, batch_y, weights):
    for feature_name in CATEGORICAL_FEATURE_NAMES:
        batch_x[feature_name] = lookup_dict[feature_name](batch_x[feature_name])

    return batch_x, batch_y, weights


def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
    dataset = (
        tf_data.experimental.make_csv_dataset(
            csv_file_path,
            batch_size=batch_size,
            column_names=CSV_HEADER,
            column_defaults=COLUMN_DEFAULTS,
            label_name=TARGET_FEATURE_NAME,
            num_epochs=1,
            header=False,
            na_value="?",
            shuffle=shuffle,
        )
        .map(prepare_example, num_parallel_calls=tf_data.AUTOTUNE, deterministic=False)
        .map(encode_categorical)
    )
    return dataset.cache()


"""
## Implement a training and evaluation procedure
"""


def run_experiment(
    model,
    train_data_file,
    test_data_file,
    num_epochs,
    learning_rate,
    weight_decay,
    batch_size,
    callbacks=None,
):
    optimizer = keras.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[keras.metrics.BinaryAccuracy(name="accuracy"),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc'),
                ],
    )

    train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
    validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

    print("Start training the model...")
    start_time = time.time()
    history = model.fit(
        train_dataset, epochs=num_epochs, validation_data=validation_dataset,callbacks=callbacks
    )
    training_time = time.time() - start_time
    print(f"Training finished in {training_time:.2f} seconds")

    print("Model training finished")
    start_time = time.time()
    loss, accuracy, precision, recall, auc  = model.evaluate(validation_dataset, verbose=0)

    print(f"Validation accuracy: {round(accuracy * 100, 2)}%")
    inference_time = time.time() - start_time
    print(f"Inference time: {inference_time:.2f} seconds")

    # Assuming your model's final layer is a sigmoid, as you're using BinaryCrossentropy
    # If it's softmax, you'll need to adapt this and use `predict_classes` or similar
    y_pred = (model.predict(validation_dataset) > 0.5).astype("int32")
    y_true = []
    for x, y, w in validation_dataset:
        y_true.extend(y.numpy())
    y_true = np.array(y_true).flatten()  # Ensure y_true is a flat array
    assert len(y_true) == len(y_pred), "Mismatch in the number of samples between y_true and y_pred"


    #print(f"Validation accuracy: {results[1]:.2f}%")
    #print(f"AUC: {results[-1]}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    # Assuming binary classification and y_true and y_pred are numpy arrays
    auc_score = roc_auc_score(y_true, y_pred)
    print(f"AUC ROC Score: {auc_score}")

    return history, training_time, inference_time


"""
## Create model inputs

Now, define the inputs for the models as a dictionary, where the key is the feature name,
and the value is a `keras.layers.Input` tensor with the corresponding feature shape
and data type.
"""


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype="float32"
            )
    return inputs


"""
## Encode features

The `encode_inputs` method returns `encoded_categorical_feature_list` and `numerical_feature_list`.
We encode the categorical features as embeddings, using a fixed `embedding_dims` for all the features,
regardless their vocabulary sizes. This is required for the Transformer model.
"""


def encode_inputs(inputs, embedding_dims):
    encoded_categorical_feature_list = []
    numerical_feature_list = []

    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert a string values to an integer indices.
            # Since we are not using a mask token, nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and num_oov_indices to 0.

            # Convert the string input values into integer indices.

            # Create an embedding layer with the specified dimensions.
            embedding = layers.Embedding(
                input_dim=len(vocabulary), output_dim=embedding_dims
            )

            # Convert the index values to embedding representations.
            encoded_categorical_feature = embedding(inputs[feature_name])
            encoded_categorical_feature_list.append(encoded_categorical_feature)

        else:
            # Use the numerical features as-is.
            numerical_feature = ops.expand_dims(inputs[feature_name], -1)
            numerical_feature_list.append(numerical_feature)

    return encoded_categorical_feature_list, numerical_feature_list


"""
## Implement an MLP block
"""


def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):
    mlp_layers = []
    for units in hidden_units:
        mlp_layers.append(normalization_layer()),
        mlp_layers.append(layers.Dense(units, activation=activation))
        mlp_layers.append(layers.Dropout(dropout_rate))

    return keras.Sequential(mlp_layers, name=name)


"""
## Experiment 1: a baseline model

In the first experiment, we create a simple multi-layer feed-forward network.
"""


def create_baseline_model(
    embedding_dims, num_mlp_blocks, mlp_hidden_units_factors, dropout_rate
):
    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )
    # Concatenate all features.
    features = layers.concatenate(
        encoded_categorical_feature_list + numerical_feature_list
    )
    # Compute Feedforward layer units.
    feedforward_units = [features.shape[-1]]

    # Create several feedforwad layers with skip connections.
    for layer_idx in range(num_mlp_blocks):
        features = create_mlp(
            hidden_units=feedforward_units,
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=layers.LayerNormalization,
            name=f"feedforward_{layer_idx}",
        )(features)

    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization,
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model(
    embedding_dims=EMBEDDING_DIMS,
    num_mlp_blocks=NUM_MLP_BLOCKS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", baseline_model.count_params())
keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

"""
Let's train and evaluate the baseline model:
"""

history = run_experiment(
    model=baseline_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
)



## Experiment 2: TabTransformer



def create_tabtransformer_classifier(
    num_transformer_blocks,
    num_heads,
    embedding_dims,
    mlp_hidden_units_factors,
    dropout_rate,
    use_column_embedding=False,
):
    # Create model inputs.
    inputs = create_model_inputs()
    # encode features.
    encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
        inputs, embedding_dims
    )


    # Stack categorical feature embeddings for the Tansformer.
    encoded_categorical_features = ops.stack(encoded_categorical_feature_list, axis=1)
    # Concatenate numerical features.
    numerical_features = layers.concatenate(numerical_feature_list)

    # Add column embedding to categorical feature embeddings.
    if use_column_embedding:
        num_columns = encoded_categorical_features.shape[1]
        column_embedding = layers.Embedding(
            input_dim=num_columns, output_dim=embedding_dims
        )
        column_indices = ops.arange(start=0, stop=num_columns, step=1)
        encoded_categorical_features = encoded_categorical_features + column_embedding(
            column_indices
        )

    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)
        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
        # Feedforward.
        feedforward_output = create_mlp(
            hidden_units=[embedding_dims],
            dropout_rate=dropout_rate,
            activation=keras.activations.gelu,
            normalization_layer=partial(
                layers.LayerNormalization, epsilon=1e-6
            ),  # using partial to provide keyword arguments before initialization
            name=f"feedforward_{block_idx}",
        )(x)
        # Skip connection 2.
        x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
        # Layer normalization 2.
        encoded_categorical_features = layers.LayerNormalization(
            name=f"layer_norm2_{block_idx}", epsilon=1e-6
        )(x)





    # Flatten the "contextualized" embeddings of the categorical features.
    categorical_features = layers.Flatten()(encoded_categorical_features)
    # Apply layer normalization to the numerical features.
    numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
    # Prepare the input for the final MLP block.
    features = layers.concatenate([categorical_features, numerical_features])



    # Create multiple layers of the Transformer block.
    for block_idx in range(num_transformer_blocks):
        # Batch Normalization before the MultiHead Attention
        encoded_categorical_features = layers.BatchNormalization()(encoded_categorical_features)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dims,
            dropout=dropout_rate,
            name=f"multihead_attention_{block_idx}",
        )(encoded_categorical_features, encoded_categorical_features)

        # Skip connection 1.
        x = layers.Add(name=f"skip_connection1_{block_idx}")(
            [attention_output, encoded_categorical_features]
        )
        # Layer normalization 1.
        x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)







    # Compute MLP hidden_units.
    mlp_hidden_units = [
        factor * features.shape[-1] for factor in mlp_hidden_units_factors
    ]
    # Create final MLP.
    features = create_mlp(
        hidden_units=mlp_hidden_units,
        dropout_rate=dropout_rate,
        activation=keras.activations.selu,
        normalization_layer=layers.BatchNormalization,
        name="MLP",
    )(features)

    # Add a sigmoid as a binary classifer.
    outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model






tabtransformer_model = create_tabtransformer_classifier(
    num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
    num_heads=NUM_HEADS,
    embedding_dims=EMBEDDING_DIMS,
    mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
    dropout_rate=DROPOUT_RATE,
)

print("Total model weights:", tabtransformer_model.count_params())
keras.utils.plot_model(tabtransformer_model, show_shapes=True, rankdir="LR")

"""
Let's train and evaluate the TabTransformer model:
"""
# Define callbacks for learning rate scheduling and early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6)

# ...
history = run_experiment(
    model=tabtransformer_model,
    train_data_file=train_data_file,
    test_data_file=test_data_file,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping_callback, reduce_lr_callback]  # Add callbacks here
)

Total model weights: 3541
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.
Start training the model...
Epoch 1/20
   6294/Unknown 15s 2ms/step - accuracy: 0.7002 - auc: 0.8001 - loss: 0.4831 - precision: 0.6744 - recall: 0.7691

2024-02-03 02:59:58.733318: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15520934327594066847
2024-02-03 02:59:58.733340: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4592894476644585425
2024-02-03 02:59:58.733348: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2198605829791602770
2024-02-03 02:59:58.733353: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4448486198308456762
2024-02-03 02:59:58.733362: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9289195203008978844
/home/irciss/anaconda3/envs/py39/lib/python3.9/contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `

6294/6294 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.7002 - auc: 0.8001 - loss: 0.4831 - precision: 0.6744 - recall: 0.7691 - val_accuracy: 0.7631 - val_auc: 0.8689 - val_loss: 0.3742 - val_precision: 0.6788 - val_recall: 0.9978
Epoch 2/20
 124/6294 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.7548 - auc: 0.8617 - loss: 0.3841 - precision: 0.7003 - recall: 0.8868

2024-02-03 02:59:59.961406: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9268208623841757565
2024-02-03 02:59:59.961423: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15520934327594066847
2024-02-03 02:59:59.961426: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4592894476644585425
2024-02-03 02:59:59.961432: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 2198605829791602770
2024-02-03 02:59:59.961434: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4448486198308456762
2024-02-03 02:59:59.961436: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10139441375753848656
2024-02-03 02:59:59.961438: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

6267/6294 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7566 - auc: 0.8629 - loss: 0.3821 - precision: 0.7000 - recall: 0.8982

...

6285/6294 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7621 - auc: 0.8691 - loss: 0.3741 - precision: 0.6941 - recall: 0.9374

2024-02-03 03:07:24.981793: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9268208623841757565
2024-02-03 03:07:24.981813: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15520934327594066847
2024-02-03 03:07:24.981816: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4592894476644585425
2024-02-03 03:07:24.981819: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10139441375753848656
2024-02-03 03:07:24.981821: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4448486198308456762
2024-02-03 03:07:24.981823: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 9289195203008978844
2024-02-03 03:07:24.981825: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv i

6294/6294 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - accuracy: 0.7621 - auc: 0.8691 - loss: 0.3741 - precision: 0.6941 - recall: 0.9374 - val_accuracy: 0.7631 - val_auc: 0.8737 - val_loss: 0.3722 - val_precision: 0.6788 - val_recall: 0.9978 - learning_rate: 1.0000e-05
Epoch 15/20
6294/6294 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.7622 - auc: 0.8692 - loss: 0.3741 - precision: 0.6942 - recall: 0.9374 - val_accuracy: 0.7631 - val_auc: 0.8737 - val_loss: 0.3722 - val_precision: 0.6788 - val_recall: 0.9978 - learning_rate: 1.0000e-06
Training finished in 308.36 seconds
Model training finished


2024-02-03 03:07:45.814288: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4592894476644585425
2024-02-03 03:07:45.814309: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4448486198308456762


Validation accuracy: 76.31%
Inference time: 0.72 seconds
787/787 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.53      0.69    104385
           1       0.68      1.00      0.81    104096

    accuracy                           0.76    208481
   macro avg       0.84      0.76      0.75    208481
weighted avg       0.84      0.76      0.75    208481

AUC ROC Score: 0.7634734413453907


In [88]:
# def plot_learning_curves(history):
#     acc = history.history['accuracy']
#     val_acc = history.history['val_accuracy']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']

#     epochs = range(1, len(acc) + 1)

#     plt.figure(figsize=(15, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(epochs, acc, 'bo', label='Training acc')
#     plt.plot(epochs, val_acc, 'b', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()

#     plt.subplot(1, 2, 2)
#     plt.plot(epochs, loss, 'bo', label='Training loss')
#     plt.plot(epochs, val_loss, 'b', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()

#     plt.show()

# history, training_time, inference_time = run_experiment(
#         model=tabtransformer_model,
#     train_data_file=train_data_file,
#     test_data_file=test_data_file,
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     weight_decay=WEIGHT_DECAY,
#     batch_size=BATCH_SIZE,
#     callbacks=[early_stopping_callback, reduce_lr_callback]
# )

# plot_learning_curves(history)

In [89]:
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau


# ## Prepare the data

# data_columns = list(data.columns)

# CSV_HEADER = data_columns


# """
# ## Define dataset metadata

# Here, we define the metadata of the dataset that will be useful for reading and parsing
# the data into input features, and encoding the input features with respect to their types.
# """

# # A list of the numerical feature names.
# NUMERIC_FEATURE_NAMES = data_columns[1:len(data_columns)-2]


# # A dictionary of the categorical features and their vocabulary.
# CATEGORICAL_FEATURES_WITH_VOCABULARY = {
#     "protocol": sorted(list(data_train["protocol"].unique())),
# }
# # Name of the column to be used as instances weight.
# WEIGHT_COLUMN_NAME = "fnlwgt"
# # A list of the categorical feature names.
# CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
# # A list of all the input features.
# FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES
# # A list of column default values for each feature.
# COLUMN_DEFAULTS = [
#     [0.0] if feature_name in NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME] else ["NA"]
#     for feature_name in CSV_HEADER
# ]
# # The name of the target feature.
# TARGET_FEATURE_NAME = "is_attack"
# # A list of the labels of the target features.
# TARGET_LABELS = ["yes", "no"]

# """
# ## Configure the hyperparameters

# The hyperparameters includes model architecture and training configurations.
# """

# LEARNING_RATE = 0.001
# WEIGHT_DECAY = 0.0001
# DROPOUT_RATE = 0.2
# BATCH_SIZE = 265
# NUM_EPOCHS = 20

# NUM_TRANSFORMER_BLOCKS = 4  # Number of transformer blocks.
# NUM_HEADS = 4  # Number of attention heads.
# EMBEDDING_DIMS = 16  # Embedding dimensions of the categorical features.
# MLP_HIDDEN_UNITS_FACTORS = [
#     2,
#     1,
# ]  # MLP hidden layer units, as factors of the number of inputs.
# NUM_MLP_BLOCKS = 2  # Number of MLP blocks in the baseline model.

# """
# ## Implement data reading pipeline

# We define an input function that reads and parses the file, then converts features
# and labels into a[`tf.data.Dataset`](https://www.tensorflow.org/guide/datasets)
# for training or evaluation.
# """

# target_label_lookup = layers.StringLookup(
#     vocabulary=TARGET_LABELS, mask_token=None, num_oov_indices=0
# )


# def prepare_example(features, target):
#     target_index = target_label_lookup(target)
#     weights = features.pop(WEIGHT_COLUMN_NAME)
#     return features, target_index, weights


# lookup_dict = {}
# for feature_name in CATEGORICAL_FEATURE_NAMES:
#     vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
#     # Create a lookup to convert a string values to an integer indices.
#     # Since we are not using a mask token, nor expecting any out of vocabulary
#     # (oov) token, we set mask_token to None and num_oov_indices to 0.
#     lookup = layers.StringLookup(
#         vocabulary=vocabulary, mask_token=None, num_oov_indices=0
#     )
#     lookup_dict[feature_name] = lookup


# def encode_categorical(batch_x, batch_y, weights):
#     for feature_name in CATEGORICAL_FEATURE_NAMES:
#         batch_x[feature_name] = lookup_dict[feature_name](batch_x[feature_name])

#     return batch_x, batch_y, weights


# def get_dataset_from_csv(csv_file_path, batch_size=128, shuffle=False):
#     dataset = (
#         tf_data.experimental.make_csv_dataset(
#             csv_file_path,
#             batch_size=batch_size,
#             column_names=CSV_HEADER,
#             column_defaults=COLUMN_DEFAULTS,
#             label_name=TARGET_FEATURE_NAME,
#             num_epochs=1,
#             header=False,
#             na_value="?",
#             shuffle=shuffle,
#         )
#         .map(prepare_example, num_parallel_calls=tf_data.AUTOTUNE, deterministic=False)
#         .map(encode_categorical)
#     )
#     return dataset.cache()


# """
# ## Implement a training and evaluation procedure
# """


# def run_experiment(
#     model,
#     train_data_file,
#     test_data_file,
#     num_epochs,
#     learning_rate,
#     weight_decay,
#     batch_size,
#     callbacks=None,
# ):
#     optimizer = keras.optimizers.AdamW(
#         learning_rate=learning_rate, weight_decay=weight_decay
#     )

#     model.compile(
#         optimizer=optimizer,
#         loss=keras.losses.BinaryCrossentropy(),
#         metrics=[keras.metrics.BinaryAccuracy(name="accuracy")],
#     )

#     train_dataset = get_dataset_from_csv(train_data_file, batch_size, shuffle=True)
#     validation_dataset = get_dataset_from_csv(test_data_file, batch_size)

#     print("Start training the model...")
#     history = model.fit(
#         train_dataset, epochs=num_epochs, validation_data=validation_dataset,callbacks=callbacks
#     )
#     print("Model training finished")

#     _, accuracy = model.evaluate(validation_dataset, verbose=0)

#     print(f"Validation accuracy: {round(accuracy * 100, 2)}%")

#     return history


# """
# ## Create model inputs

# Now, define the inputs for the models as a dictionary, where the key is the feature name,
# and the value is a `keras.layers.Input` tensor with the corresponding feature shape
# and data type.
# """


# def create_model_inputs():
#     inputs = {}
#     for feature_name in FEATURE_NAMES:
#         if feature_name in NUMERIC_FEATURE_NAMES:
#             inputs[feature_name] = layers.Input(
#                 name=feature_name, shape=(), dtype="float32"
#             )
#         else:
#             inputs[feature_name] = layers.Input(
#                 name=feature_name, shape=(), dtype="float32"
#             )
#     return inputs


# """
# ## Encode features

# The `encode_inputs` method returns `encoded_categorical_feature_list` and `numerical_feature_list`.
# We encode the categorical features as embeddings, using a fixed `embedding_dims` for all the features,
# regardless their vocabulary sizes. This is required for the Transformer model.
# """


# def encode_inputs(inputs, embedding_dims):
#     encoded_categorical_feature_list = []
#     numerical_feature_list = []

#     for feature_name in inputs:
#         if feature_name in CATEGORICAL_FEATURE_NAMES:
#             vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
#             # Create a lookup to convert a string values to an integer indices.
#             # Since we are not using a mask token, nor expecting any out of vocabulary
#             # (oov) token, we set mask_token to None and num_oov_indices to 0.

#             # Convert the string input values into integer indices.

#             # Create an embedding layer with the specified dimensions.
#             embedding = layers.Embedding(
#                 input_dim=len(vocabulary), output_dim=embedding_dims
#             )

#             # Convert the index values to embedding representations.
#             encoded_categorical_feature = embedding(inputs[feature_name])
#             encoded_categorical_feature_list.append(encoded_categorical_feature)

#         else:
#             # Use the numerical features as-is.
#             numerical_feature = ops.expand_dims(inputs[feature_name], -1)
#             numerical_feature_list.append(numerical_feature)

#     return encoded_categorical_feature_list, numerical_feature_list


# """
# ## Implement an MLP block
# """


# def create_mlp(hidden_units, dropout_rate, activation, normalization_layer, name=None):
#     mlp_layers = []
#     for units in hidden_units:
#         mlp_layers.append(normalization_layer()),
#         mlp_layers.append(layers.Dense(units, activation=activation))
#         mlp_layers.append(layers.Dropout(dropout_rate))

#     return keras.Sequential(mlp_layers, name=name)


# """
# ## Experiment 1: a baseline model

# In the first experiment, we create a simple multi-layer feed-forward network.
# """


# def create_baseline_model(
#     embedding_dims, num_mlp_blocks, mlp_hidden_units_factors, dropout_rate
# ):
#     # Create model inputs.
#     inputs = create_model_inputs()
#     # encode features.
#     encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
#         inputs, embedding_dims
#     )
#     # Concatenate all features.
#     features = layers.concatenate(
#         encoded_categorical_feature_list + numerical_feature_list
#     )
#     # Compute Feedforward layer units.
#     feedforward_units = [features.shape[-1]]

#     # Create several feedforwad layers with skip connections.
#     for layer_idx in range(num_mlp_blocks):
#         features = create_mlp(
#             hidden_units=feedforward_units,
#             dropout_rate=dropout_rate,
#             activation=keras.activations.gelu,
#             normalization_layer=layers.LayerNormalization,
#             name=f"feedforward_{layer_idx}",
#         )(features)

#     # Compute MLP hidden_units.
#     mlp_hidden_units = [
#         factor * features.shape[-1] for factor in mlp_hidden_units_factors
#     ]
#     # Create final MLP.
#     features = create_mlp(
#         hidden_units=mlp_hidden_units,
#         dropout_rate=dropout_rate,
#         activation=keras.activations.selu,
#         normalization_layer=layers.BatchNormalization,
#         name="MLP",
#     )(features)

#     # Add a sigmoid as a binary classifer.
#     outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
#     model = keras.Model(inputs=inputs, outputs=outputs)
#     return model


# baseline_model = create_baseline_model(
#     embedding_dims=EMBEDDING_DIMS,
#     num_mlp_blocks=NUM_MLP_BLOCKS,
#     mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
#     dropout_rate=DROPOUT_RATE,
# )

# print("Total model weights:", baseline_model.count_params())
# keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

# """
# Let's train and evaluate the baseline model:
# """

# history = run_experiment(
#     model=baseline_model,
#     train_data_file=train_data_file,
#     test_data_file=test_data_file,
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     weight_decay=WEIGHT_DECAY,
#     batch_size=BATCH_SIZE,
# )



# ## Experiment 2: TabTransformer



# def create_tabtransformer_classifier(
#     num_transformer_blocks,
#     num_heads,
#     embedding_dims,
#     mlp_hidden_units_factors,
#     dropout_rate,
#     use_column_embedding=False,
# ):
#     # Create model inputs.
#     inputs = create_model_inputs()
#     # encode features.
#     encoded_categorical_feature_list, numerical_feature_list = encode_inputs(
#         inputs, embedding_dims
#     )


#     # Stack categorical feature embeddings for the Tansformer.
#     encoded_categorical_features = ops.stack(encoded_categorical_feature_list, axis=1)
#     # Concatenate numerical features.
#     numerical_features = layers.concatenate(numerical_feature_list)

#     # Add column embedding to categorical feature embeddings.
#     if use_column_embedding:
#         num_columns = encoded_categorical_features.shape[1]
#         column_embedding = layers.Embedding(
#             input_dim=num_columns, output_dim=embedding_dims
#         )
#         column_indices = ops.arange(start=0, stop=num_columns, step=1)
#         encoded_categorical_features = encoded_categorical_features + column_embedding(
#             column_indices
#         )

#     # Create multiple layers of the Transformer block.
#     for block_idx in range(num_transformer_blocks):
#         # Create a multi-head attention layer.
#         attention_output = layers.MultiHeadAttention(
#             num_heads=num_heads,
#             key_dim=embedding_dims,
#             dropout=dropout_rate,
#             name=f"multihead_attention_{block_idx}",
#         )(encoded_categorical_features, encoded_categorical_features)
#         # Skip connection 1.
#         x = layers.Add(name=f"skip_connection1_{block_idx}")(
#             [attention_output, encoded_categorical_features]
#         )
#         # Layer normalization 1.
#         x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)
#         # Feedforward.
#         feedforward_output = create_mlp(
#             hidden_units=[embedding_dims],
#             dropout_rate=dropout_rate,
#             activation=keras.activations.gelu,
#             normalization_layer=partial(
#                 layers.LayerNormalization, epsilon=1e-6
#             ),  # using partial to provide keyword arguments before initialization
#             name=f"feedforward_{block_idx}",
#         )(x)
#         # Skip connection 2.
#         x = layers.Add(name=f"skip_connection2_{block_idx}")([feedforward_output, x])
#         # Layer normalization 2.
#         encoded_categorical_features = layers.LayerNormalization(
#             name=f"layer_norm2_{block_idx}", epsilon=1e-6
#         )(x)





#     # Flatten the "contextualized" embeddings of the categorical features.
#     categorical_features = layers.Flatten()(encoded_categorical_features)
#     # Apply layer normalization to the numerical features.
#     numerical_features = layers.LayerNormalization(epsilon=1e-6)(numerical_features)
#     # Prepare the input for the final MLP block.
#     features = layers.concatenate([categorical_features, numerical_features])



#     # Create multiple layers of the Transformer block.
#     for block_idx in range(num_transformer_blocks):
#         # Batch Normalization before the MultiHead Attention
#         encoded_categorical_features = layers.BatchNormalization()(encoded_categorical_features)

#         # Create a multi-head attention layer.
#         attention_output = layers.MultiHeadAttention(
#             num_heads=num_heads,
#             key_dim=embedding_dims,
#             dropout=dropout_rate,
#             name=f"multihead_attention_{block_idx}",
#         )(encoded_categorical_features, encoded_categorical_features)

#         # Skip connection 1.
#         x = layers.Add(name=f"skip_connection1_{block_idx}")(
#             [attention_output, encoded_categorical_features]
#         )
#         # Layer normalization 1.
#         x = layers.LayerNormalization(name=f"layer_norm1_{block_idx}", epsilon=1e-6)(x)







#     # Compute MLP hidden_units.
#     mlp_hidden_units = [
#         factor * features.shape[-1] for factor in mlp_hidden_units_factors
#     ]
#     # Create final MLP.
#     features = create_mlp(
#         hidden_units=mlp_hidden_units,
#         dropout_rate=dropout_rate,
#         activation=keras.activations.selu,
#         normalization_layer=layers.BatchNormalization,
#         name="MLP",
#     )(features)

#     # Add a sigmoid as a binary classifer.
#     outputs = layers.Dense(units=1, activation="sigmoid", name="sigmoid")(features)
#     model = keras.Model(inputs=inputs, outputs=outputs)
#     return model






# tabtransformer_model = create_tabtransformer_classifier(
#     num_transformer_blocks=NUM_TRANSFORMER_BLOCKS,
#     num_heads=NUM_HEADS,
#     embedding_dims=EMBEDDING_DIMS,
#     mlp_hidden_units_factors=MLP_HIDDEN_UNITS_FACTORS,
#     dropout_rate=DROPOUT_RATE,
# )

# print("Total model weights:", tabtransformer_model.count_params())
# keras.utils.plot_model(tabtransformer_model, show_shapes=True, rankdir="LR")

# """
# Let's train and evaluate the TabTransformer model:
# """
# # Define callbacks for learning rate scheduling and early stopping
# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6)

# # ...
# history = run_experiment(
#     model=tabtransformer_model,
#     train_data_file=train_data_file,
#     test_data_file=test_data_file,
#     num_epochs=NUM_EPOCHS,
#     learning_rate=LEARNING_RATE,
#     weight_decay=WEIGHT_DECAY,
#     batch_size=BATCH_SIZE,
#     callbacks=[early_stopping_callback, reduce_lr_callback]  # Add callbacks here
# # )